In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

C:\MAD4AG


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import geopandas

import warnings

warnings.filterwarnings('ignore')

C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
file_name = f'./dbs/intermediate/stops_1_new.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)
# df = df[df.holiday_s != 1]
# df = df[df.weekday_s == 1]
# df = df.drop(['holiday_s', 'weekday_s'], axis=1)
df.drop_duplicates(subset=['uid', 'cluster'], keep='first', inplace=True)

In [4]:
df_h = pd.read_parquet(f'./dbs/intermediate/home_inference.parquet')
df_h.drop_duplicates(subset='uid', keep='first', inplace=True)
df_h['home_potential'] = 1
df_h['work_potential'] = 0

In [5]:
df_w = pd.read_parquet(f'./dbs/intermediate/work_inference.parquet')
df_w.drop_duplicates(subset='uid', keep='first', inplace=True)
df_w['home_potential'] = 0

ppl_having_home = df_h.uid.unique()
df_w = df_w[df_w.uid.isin(ppl_having_home)]

In [6]:
df_act = pd.concat([df_h[['uid', 'cluster', 'home_potential', 'work_potential']], df_w[['uid', 'cluster', 'home_potential', 'work_potential']]],axis=0, ignore_index=True).sort_values(by='uid')

In [7]:
df_act = pd.merge(df_act, df[['uid', 'cluster', 'cluster_lat', 'cluster_lng' ]], on=['uid', 'cluster'], how="left")


In [8]:
DeSO = geopandas.read_file(
    f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/deso_statistik_shp/Bef_Alder_region.shp')

DeSO.loc[:, 'adult_pop'] = DeSO.loc[:, [ 'Ald20_24', 'Ald25_29', 'Ald30_34', 'Ald35_39', 'Ald40_44', 'Ald45_49', 'Ald50_54', 'Ald55_59', 'Ald60_64', 'Ald65_69', 'Ald70_74', 'Ald75_79', 'Ald80_w']].sum(axis=1) + DeSO.loc[:,'Ald16_19']/2

DeSO_arb = geopandas.read_file(
    f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/deso_statistik_shp/Bef_Sysselsattning_region.shp')

DeSO = pd.merge(DeSO, DeSO_arb[['Deso','Forvarb']], left_on=['Deso'], right_on=['Deso'], how="left")
DeSO.rename(columns={'Forvarb':'employee_pop'}, inplace=True )

print(DeSO.crs)

DeSO.to_crs(4326, inplace=True)
print(DeSO.crs)

PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWEREF99",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6619"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3006"]]
epsg:4326


In [9]:
gdf_act = geopandas.GeoDataFrame(df_act, geometry=geopandas.points_from_xy(df_act.cluster_lng, df_act.cluster_lat), crs="EPSG:4326")

In [10]:
gdf_act = gdf_act.sjoin(DeSO[['Deso','geometry', 'adult_pop','employee_pop']], how="left")


In [11]:
gdf_2_home = gdf_act[gdf_act.home_potential== 1]

gdf_2_home= gdf_2_home.groupby('Deso')['uid'].count().reset_index().rename(columns={"uid":'home_count'})

In [12]:
# gdf_2_home.home_count.sum()

In [13]:
gdf_2_home = pd.merge(DeSO[['Deso','adult_pop', 'employee_pop']], gdf_2_home, left_on=['Deso'], right_on=['Deso'], how="left")

In [14]:
gdf_2_home.loc[:, 'wt_p'] = gdf_2_home.loc[:, 'adult_pop'] / gdf_2_home.loc[:, 'home_count']


In [15]:
w0 = ((np.std(gdf_2_home.loc[:, 'wt_p']) / np.mean(gdf_2_home.loc[:, 'wt_p'])) ** 2 + 1) ** 0.5 * 3.5 * np.median(
    gdf_2_home.loc[:, 'wt_p'])

In [16]:
gdf_2_home.loc[gdf_2_home['wt_p'] > w0, 'wt_p'] = w0

np.sum(gdf_2_home['wt_p'] * gdf_2_home['home_count'])

8057544.306518847

In [17]:
employees = df_act['uid'][df_act.work_potential== 1].unique()


In [18]:
gdf_act = gdf_act[gdf_act.home_potential == 1]
gdf_act['work_potential'][gdf_act.uid.isin(employees)] = 1

In [19]:
gdf_act = pd.merge(gdf_act, gdf_2_home[['Deso','wt_p']], on='Deso', how='left')

In [20]:
df_act = gdf_act[['uid', 'Deso', 'work_potential', 'adult_pop', 'employee_pop', 'wt_p']].sort_values(by=['Deso'])

In [21]:
def ipf_wtp(data):
    current_pop = data['wt_p'].sum()
    #pop_factor = current_pop/data['adult_pop']
    for it in range(20):

        employees = data['wt_p'][data['work_potential']==1].sum() #* pop_factor
        data['wt_p'][data['work_potential']==1] = data['wt_p'][data['work_potential']==1] * (data['employee_pop']/ employees)
        population = data['wt_p'].sum()
        data['wt_p'] = data['wt_p'] * (current_pop/ population)
    return data

In [22]:
tqdm.pandas()
df_act = df_act.groupby('Deso').progress_apply(ipf_wtp)

  0%|          | 0/5985 [00:00<?, ?it/s]

# instead of the number of employees from deso, I should use the work activity daily participation ratio. The activity participation can be calculated from the travel survey.

## in the script 1.1, use an algorithm looking within the sweden polygone instead of spatial join with deso
## in the script 1.1, directly remove the holidays and save the ready data
## in the script 1.1, save the building locations as lat long as before